In [ ]:
from ultralytics import YOLO
import cv2          # 匯入 OpenCV
import numpy as np  # 匯入 NumPy
import os           # 匯入 os 模組

: 

In [ ]:
from ultralytics import YOLO
import cv2          # 匯入 OpenCV
import numpy as np  # 匯入 NumPy
import os           # 匯入 os 模組

# 載入您自訂的分割模型
model = YOLO('../use_model/yolo11m-seg_cow/weights/best.pt')

# 影片檔案路徑
video_path = '/mnt/z/cattle_temp_data/智農計畫-子8-跛行/dataset-for-unknow/video/LV1/403/lameness_001_clip_00-14_to_00-19.mp4'

# --- 1. 建立儲存遮罩的輸出資料夾 ---
output_dir = "result_masks"
os.makedirs(output_dir, exist_ok=True)

# --- 2. 執行推論 ---
# 對於影片，建議使用 stream=True 模式以節省記憶體，它會回傳一個結果生成器
results_generator = model(video_path, 
                          imgsz=640, 
                          iou=0.9, 
                          conf=0.5,
                          device='cuda:0',  # 使用 CUDA 裝置
                          stream=True
                          ) 

# --- 3. 處理結果並儲存每個獨立的遮罩 ---
print(f"開始處理影片: {video_path}")
print(f"遮罩將儲存於 '{output_dir}/' 資料夾中...")

# 使用 enumerate 獲取畫格的索引 (frame_idx)
for frame_idx, result in enumerate(results_generator):
    # 檢查該畫格是否有偵測到遮罩
    if result.masks is not None:
        # result.masks.data 是一個 (N, H, W) 的張量，N 是該畫格偵測到的物件數量
        # 迭代處理該畫格中的每一個物件遮罩
        for obj_idx, mask_tensor in enumerate(result.masks.data):
            
            # 將遮罩張量轉換為 NumPy 陣列，資料格式為 uint8
            # .cpu() 是為了確保張量在 CPU 上，以便轉換為 NumPy 陣列
            binary_mask = mask_tensor.cpu().numpy().astype(np.uint8)
            
            # 將遮罩的數值從 [0, 1] 轉換為 [0, 255]，以符合黑白影像格式
            binary_mask *= 255
            
            # 建立一個獨一無二的檔案名稱，格式為：frame_{畫格編號}_object_{物件編號}.png
            filename = os.path.join(output_dir, f"frame_{frame_idx:04d}_object_{obj_idx:02d}.png")
            
            # 使用 OpenCV 將 NumPy 陣列儲存為 PNG 檔案
            cv2.imwrite(filename, binary_mask)

print("處理完成。")

: 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# 載入分割模型
model = YOLO('../use_model/yolo11m-seg_cow/weights/best.pt')

# 開啟影片檔案
# 請將 'your_video.mp4' 替換成您的影片路徑
video_path = '/mnt/z/cattle_temp_data/智農計畫-子8-跛行/dataset-for-unknow/video/LV1/403/lameness_001_clip_00-14_to_00-19.mp4'
cap = cv2.VideoCapture(video_path)

# 確認影片是否成功開啟
if not cap.isOpened():
    print("錯誤: 無法開啟影片檔案。")
    exit()

# 迴圈處理影片的每一個畫格
while cap.isOpened():
    # 讀取一個畫格
    success, frame = cap.read()

    if success:
        # 使用 YOLOv8 進行推論，設定 stream=True 以獲得結果生成器
        results = model(frame, stream=True)

        # 處理每個畫格的結果
        for result in results:
            # 檢查是否偵測到任何遮罩
            if result.masks is not None:
                # result.masks.data 是一個 (N, H, W) 的張量
                # N 是偵測到的物件數量
                # 我們可以迭代取得每一個物件的獨立遮罩
                for i, mask_tensor in enumerate(result.masks.data):
                    # 將遮罩張量轉換為 NumPy 陣列
                    # 遮罩的數值為 0 (背景) 或 1 (物件)，我們將其乘以 255 轉換為可視化的灰階影像
                    binary_mask = mask_tensor.cpu().numpy().astype(np.uint8) * 255

                    # binary_mask 就是第 i 個物件的獨立二元遮罩
                    # 您可以在這裡對這個遮罩進行後續處理
                    # 例如：儲存成檔案、計算面積、或是將其應用於原影像上

                    # 顯示每個獨立的遮罩 (可選)
                    mask_display_name = f'Object {i+1} Mask'
                    cv2.imshow(mask_display_name, binary_mask)

                    # 您也可以提取物件的邊界框和類別
                    box = result.boxes[i]
                    class_id = int(box.cls[0])
                    class_name = model.names[class_id]
                    print(f"物件 {i+1}: 類別 = {class_name}")


        # 顯示帶有標註的原影像 (可選)
        annotated_frame = results[0].plot() # results[0] 是為了觸發生成器
        cv2.imshow("YOLO11 Inference", annotated_frame)

        # 如果按下 'q' 鍵則中斷迴圈
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # 影片結束，中斷迴圈
        break

# 釋放資源並關閉所有視窗
cap.release()
cv2.destroyAllWindows()

: 